In [5]:
# ESTE ES EL CODIGO PARA SACAR LAS MEDALLAS DE 2024

import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {"User-Agent": "Mozilla/5.0"}
url_2024 = "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Par%C3%ADs_2024"

try:
    response = requests.get(url_2024, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find("table", class_="wikitable")
    datos_paris = []

    if table:
        for row in table.find_all("tr")[1:]:
            cols = row.find_all(["td", "th"])

            if len(cols) >= 6:
                pais = cols[1].get_text(strip=True)

                # Saltar fila Total
                if pais == "Total":
                    continue

                oro = int(cols[2].get_text(strip=True))
                plata = int(cols[3].get_text(strip=True))
                bronce = int(cols[4].get_text(strip=True))
                total = int(cols[5].get_text(strip=True))

                datos_paris.append({
                    "País": pais,
                    "Oro": oro,
                    "Plata": plata,
                    "Bronce": bronce,
                    "Total_medallas": total
                })

    df_paris = pd.DataFrame(datos_paris)

    df_paris.to_csv("medallero_paris_2024.csv", index=False, encoding="utf-8")

    print("Medallero de París 2024 guardado correctamente")
    print(df_paris.head())

except Exception as e:
    print(f"Error al procesar: {e}")


Medallero de París 2024 guardado correctamente
                  País  Oro  Plata  Bronce  Total_medallas
0  Estados Unidos(USA)   40     44      42             126
1           China(CHN)   40     27      24              91
2           Japón(JPN)   20     12      13              45
3       Australia(AUS)   18     19      16              53
4         Francia(FRA)   16     26      22              64


In [1]:
#CON LA AYUDA DE OPENREFINE, HEMOS ELIMINADO ALGUNOS PAISES QUE NO SON CONSIDERADOS POR EL BANCO MUNDIAL.

In [ ]:
import pandas as pd

df = pd.read_csv("medallero-paris-FILTRADO.csv")
df["Pais"] = df["Pais"].str.split("(").str[0]       # Parte antes del paréntesis
df["ISO3"] = df["Pais"].str.split("(").str[1].str.replace(")", "")  # Parte dentro del paréntesis
df = df.drop(columns=["Pais"])
df.to_csv("medallero-paris-FILTRADO.csv", index=False)
print(df.head())

In [17]:
nombres_paises = []
for pais in df['ISO3']:
    nombres_paises.append(pais)
print(nombres_paises)

['USA', 'CHN', 'JPN', 'AUS', 'FRA', 'NLD', 'GBR', 'KOR', 'ITA', 'DEU', 'NZL', 'CAN', 'UZB', 'HUN', 'ESP', 'SWE', 'KEN', 'NOR', 'IRL', 'BRA', 'IRN', 'UKR', 'ROU', 'GEO', 'BEL', 'BGR', 'SRB', 'CZE', 'DNK', 'AZE', 'HRV', 'CUB', 'BHR', 'SVN', 'AUT', 'PHL', 'HKG', 'DZA', 'IDN', 'ISR', 'POL', 'KAZ', 'JAM', 'ZAF', 'THA', 'ETH', 'CHE', 'ECU', 'PRT', 'GRC', 'ARG', 'EGY', 'TUN', 'BWA', 'CHL', 'LCA', 'UGA', 'DOM', 'GTM', 'MAR', 'DMA', 'PAK', 'TUR', 'MEX', 'ARM', 'COL', 'KGZ', 'LTU', 'IND', 'MDA', 'CYP', 'FJI', 'JOR', 'MNG', 'PAN', 'TJK', 'ALB', 'GRD', 'MYS', 'PRI', 'CPV', 'QAT', 'CIV', 'SVK', 'PER', 'SGP', 'ZMB']


In [ ]:
import requests
import pandas as pd
# PIb: Capacidad económica del país, más inversión deportiva, centros de alto rendimiento, entrenadores
YEAR = 2024
pib = []
# La url de la api sigue la estructura https://api.worldbank.org/v2/country/(el codigo de arriba)/indicator/(el codigo del dato economico
# por ejemplo para PIB es NY.GDP.MKTP.CD?format = json(en json que es lo que hemos trabajado en clase) $ date= 2024 (que es la fecha de la cual
# queremos los datos)
for pais in nombres_paises:
    url = f"https://api.worldbank.org/v2/country/{pais}/indicator/NY.GDP.MKTP.CD?format=json&date={YEAR}"
    r = requests.get(url)
    try:
        reg = r.json()[1][0]
        if reg["value"] is not None:
            pib.append({
                "ISO3": pais,
                "PIB": reg["value"]
            })
    except:
        pass
df_pib = pd.DataFrame(pib)

In [20]:
datos_pcap = []
# PIB por capita: Nivel medio de riqueza
for pais in nombres_paises:
    url = f"https://api.worldbank.org/v2/country/{pais}/indicator/NY.GDP.PCAP.CD?format=json&date={YEAR}"
    r = requests.get(url)
    try:
        reg = r.json()[1][0]
        if reg["value"] is not None:
            datos_pcap.append({
                "ISO3": pais,
                "PIB_per_capita": reg["value"]
            })
    except:
        pass

df_pcap = pd.DataFrame(datos_pcap)

In [21]:
datos_pop = []
# Población: Más población → más atletas potenciales
for pais in nombres_paises:
    url = f"https://api.worldbank.org/v2/country/{pais}/indicator/SP.POP.TOTL?format=json&date={YEAR}"
    r = requests.get(url)
    try:
        reg = r.json()[1][0]
        if reg["value"] is not None:
            datos_pop.append({
                "ISO3": pais,
                "Poblacion": reg["value"]
            })
    except:
        pass

df_pop = pd.DataFrame(datos_pop)

In [38]:
df_final = df_pib \
    .merge(df_pcap, on="ISO3", how="inner") \
    .merge(df_pop, on="ISO3", how="inner") \
    

df_final.to_csv("economia_olimpica_2024.csv", index=False, encoding="utf-8")

print(" Dataset económico olímpico creado")
print(" Países:", len(df_final)) # Algun país no tiene datos de todo, por lo que se eliminará ese pais ya que no hay datos

 Dataset económico olímpico creado
 Países: 86


In [46]:
df1 = pd.read_csv("medallero-paris-FILTRADO.csv")  # tiene ISO3
df2 = pd.read_csv("economia_olimpica_2024.csv")            # también tiene ISO3

In [48]:
for iso in df1["ISO3"]:
    if iso in df2["ISO3"].values:
        pass
    else:
        print(f"{iso}: no hay dato en df2")
# Como se puede ver no hay datos de cuba por lo que juntamos los dos csv obviando cuba

CUB: no hay dato en df2


In [52]:
csv_final = pd.merge(df1, df2, on="ISO3", how="inner")# el inner hace que se pongan solo los que coinciden
csv_final.to_csv("csv_final.csv", index=False) 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

df = pd.read_csv('csv_final.csv')
dist = {}
    
url = 'https://www.distancefromto.net/'
driver = webdriver.Edge()
driver.maximize_window()
    
driver.get(url)
    
cookies = driver.find_element(By.CLASS_NAME, 'fc-button-label')
cookies.click()

final = driver.find_element(By.ID, 'distanceto')
final.click()
final.send_keys('Paris')

for pais in df['ISO3']:
    
    inicio = driver.find_element(By.ID, 'distancefrom')
    inicio.click()
    inicio.clear()
    inicio.send_keys(pais)
        
    buscar = driver.find_element(By.ID, 'hae')
    buscar.click()
    
    time.sleep(3)
    
    res = driver.find_element(By.ID, 'totaldistancekm')
    resultado = res.get_attribute('value')
    dist[pais] = resultado
    
    time.sleep(3)

print(dist)

df['Distancia'] = df['ISO3'].map(dist)
df.to_csv('csv_final.csv', index=False)

In [ ]:
# Ademas con este acceso a más paginas web, tambien podemos hacer un analisis exhaustivo del rendimiento de los paises durante este siglo

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

headers = {"User-Agent": "Mozilla/5.0"}

# URLs específicas de cada medallero
urls = {
    2000: "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_S%C3%ADdney_2000",
    2004: "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Atenas_2004",
    2008: "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Pek%C3%ADn_2008",
    2012: "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Londres_2012",
    2016: "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_R%C3%ADo_de_Janeiro_2016",
    2020: "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Tokio_2020",
    2024: "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Par%C3%ADs_2024"
}

# Diccionario acumulador
medallero_total = {}

# Función para convertir texto de número que puede tener puntos o espacios
def parse_int(text):
    text = re.sub(r"[^\d]", "", text)
    return int(text) if text.isdigit() else 0

for year, url in urls.items():
    print(f"Procesando medallero de {year}...")
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # Localiza la tabla (la primera tabla tipo "wikitable")
    table = soup.find("table", class_="wikitable")

    if table is None:
        print(f"No se encontró tabla para {year}")
        continue

    # Recolectar medallas por país de esa edición
    for row in table.find_all("tr")[1:]:
        cols = row.find_all("td")
        if len(cols) >= 5:
            # Normalmente en estas páginas:
            # cols[1] → país
            # cols[2] → oro
            # cols[3] → plata
            # cols[4] → bronce
            pais = cols[1].get_text(strip=True)
            pais = re.sub(r"\[.*?\]", "", pais)  # limpia referencias

            oro = parse_int(cols[2].get_text())
            plata = parse_int(cols[3].get_text())
            bronce = parse_int(cols[4].get_text())

            # Inicializa si no existe
            if pais not in medallero_total:
                medallero_total[pais] = {"Oro": 0, "Plata": 0, "Bronce": 0}

            medallero_total[pais]["Oro"] += oro
            medallero_total[pais]["Plata"] += plata
            medallero_total[pais]["Bronce"] += bronce

# Convertir a DataFrame final
df_medallas = pd.DataFrame.from_dict(medallero_total, orient="index")
df_medallas["Total"] = df_medallas["Oro"] + df_medallas["Plata"] + df_medallas["Bronce"]
df_medallas.reset_index(inplace=True)
df_medallas.rename(columns={"index": "País"}, inplace=True)

# Guardar CSV
df_medallas.to_csv("medallas_2000_2024.csv", index=False, encoding="utf-8")

print("Medallero histórico 2000–2024 guardado en medallas_2000_2024.csv")
print("Países extraídos:", len(df_medallas))
